# Section 1: Preprocessing
+ In this section, we deal with all preprocessing steps required for the rest of this notebook, including importing libraries, installing necessary packages, initializing client module for Google BigQuery etc.

In [1]:
# Path checking
%pwd

'/kaggle/working'

In [2]:
# Install google-cloud-bigquery-storage for running BigQuery SQL without error.
!pip install google-cloud-bigquery-storage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [3]:
# Import all libraries for this project
import pandas as pd

from google.cloud import bigquery, storage
from datetime import datetime, timedelta

In [4]:
# Initialize BigQuery client  
client = bigquery.Client(project='analog-delight-470708-d0')

In [5]:
# testing!
query = """
select `App ID`, Name, `Short Description` 
from `analog-delight-470708-d0.steam.steam_game_list`
where `App ID` = '1032020'; 
"""
df = client.query(query).to_dataframe()

In [6]:
df

,App ID,Name,Short Description
0,1032020,Shantalia and Corali'hulu,"A short and linear story. Every 20 years, a ma..."
